In [1]:
import re
import shutil
from os import path
from glob import glob
import yaml

In [ ]:
def get_class_name(classes: set[str]) -> str | None:
    if "cat" in classes and "prey" not in classes:
        return "no_prey"
    elif "cat" in classes and "prey" in classes:
        return "prey"
    elif len(classes) == 0:
        return "nothing"


def get_class_name_bin(classes: set[str]) -> str | None:
    if "prey" in classes:
        return "close"

    return "open"

In [ ]:
sourceDir = './datasets/source'
outputDir = './datasets/processed_binary'
labelsDirName = 'labels'
get_name = get_class_name_bin

In [3]:
with open(sourceDir+'/data.yaml') as f:
    datafile = yaml.safe_load(f)

In [4]:
datafile

{'train': '../train/images',
 'val': '../valid/images',
 'test': '../test/images',
 'nc': 2,
 'names': ['cat', 'prey'],
 'roboflow': {'workspace': 'workspac-bkbhj',
  'project': 'cat-with-prey-detection-hegrj',
  'version': 1,
  'license': 'CC BY 4.0',
  'url': 'https://universe.roboflow.com/workspac-bkbhj/cat-with-prey-detection-hegrj/dataset/1'}}

In [5]:
classNames = datafile['names']

In [ ]:
from os import makedirs


classRegexp = re.compile(r"^(\d+) ")

for imagePath in glob(sourceDir+"/**/*.jpg", recursive=True):
    labelPath = path.abspath(
        f"{path.dirname(imagePath)}/../{labelsDirName}/{path.splitext(path.basename(imagePath))[0]}.txt"
    )

    with open(labelPath, 'r') as labelFile:
        classes = set()

        for line in labelFile:
            matches = classRegexp.findall(line)

            classes.add(classNames[int(matches[0])])

    className = get_name(classes)

    if className is not None:
        classDir = f"{outputDir}/{className}"
        makedirs(classDir, exist_ok=True)
        shutil.copy(imagePath, f"{classDir}/{path.basename(imagePath)}")
    else:
        print(f"Warn: no class matched for file {path.basename(imagePath)} with label file {labelPath} . Detected classes: {classes}")
